(Example from the Book p.29 ff)

A lighthouse is somewhere off a piece of straight coastline at a position α (alpha) along the shore and a distance <br>
β (beta) out at sea. It emits a series of short highly collimated flashes at random intervals and hence at random azimuths. <br>
These pulses are intercepted on the coast by photo-detectors that record only the fact that a flash has occurred, but not <br>
the angle from which it came. <br>
N flashes have so far been recorded at positions {xk}. Where is the lighthouse?’

<img src="pictures01/lightHouseGeometry.jpg">

In [8]:
# generate data to analyze:
import numpy as np

# position of light-house on x-axis:
alpha = 1

# position of light-house off shore:
beta = 4

def data_from_angle(thetas, alpha, beta):
    return beta * np.tan(thetas) + alpha

def generate_data(numberOfDataPoints = 10):
    return np.random.uniform(size=numberOfDataPoints, low=-np.pi/2, high=np.pi/2)

# test functions:
theta_ks = generate_data()
print(theta_ks)

x_ks = data_from_angle(thetas=theta_ks, alpha=alpha, beta=beta)
print(x_ks)


[ 0.49629109  0.35448695  0.45865123 -0.02653749 -0.82856527  0.41794044
  0.15465916  1.15119131 -0.04743803 -1.12238909]


TypeError: data_from_angle() got an unexpected keyword argument 'theta'

-3.141592653589793